# advent of code 2024 - [day 9](https://adventofcode.com/2024/day/9)

### Parsing

In [10]:
def gen_data(filename='input.txt'):
    """Generates tuples of integers"""
    file = open(filename, 'r')
    line = file.readline().strip()
    for ix, c in enumerate(line.strip()):
        yield ix, "File" if ix%2 == 0 else "FreeSpace", int(c)

In [11]:
from functools import reduce
import doctest

In [77]:
filename = "input.txt"
#filename = "test.txt"

## Python Solution

## Graph Solution

In [13]:
import os

NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']

from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

import pandas as pd

def infer (rules, params={}):
    """
    This is a function you can use if you want to run a set of inference rules
    until a convergence is reached. why not use it in a RDF-like reasoning context?
    """
    counter = 0
    while True:
        counter += 1
        any_update = False
        for rule in rules:
            with driver.session(database="neo4j") as session:
                result = session.run(rule, params)
            any_update = any_update or result.consume().counters._contains_updates
        if not any_update:
            break

2.13.0


In [14]:
# cleaning
def clean():
    queries = [
    "CALL apoc.schema.assert({},{});",
    """MATCH (n)
    CALL {WITH n DETACH DELETE n}
    IN TRANSACTIONS OF 1000 ROWS;""",
    """CALL gds.graph.list()
    YIELD graphName
    WITH graphName AS g
    CALL (g) {CALL gds.graph.drop(g) YIELD graphName RETURN graphName}
    WITH graphName RETURN graphName;"""]

    for q in queries:
        gds.run_cypher(q, {})

### Ingestion


In [39]:
def ingest(filename):

    block_seq_list = list({'ix':ix, 'block_type':block_type, 'len':len} for ix, block_type, len in gen_data(filename))
    
    clean()

    gds.run_cypher('CREATE INDEX block_seq_ix IF NOT EXISTS FOR (bs:BlockSeq) ON (bs.ix)')
    gds.run_cypher('CREATE INDEX freespace_ix IF NOT EXISTS FOR (bs:FreeSpace) ON (bs.ix)')
    gds.run_cypher('CREATE INDEX file_ix IF NOT EXISTS FOR (bs:File) ON (bs.ix)')

    queries = [
    """
    UNWIND $block_seq_list AS block_seq
    CREATE (bs:BlockSeq {ix: block_seq.ix} )
    SET bs.len = block_seq.len, bs:$(block_seq.block_type)
    """,
    """
    MATCH (bs:BlockSeq)
    WITH bs ORDER BY bs.ix
    WITH collect(bs) AS bss
    CALL apoc.nodes.link(bss, "NEXT_BLOCK_SEQ")
    """,
    """
    MATCH (bs:BlockSeq&File)
    SET bs.id_number = bs.ix/2
    """,
    """MATCH (bs:BlockSeq)
    WHERE NOT EXISTS {()-[:NEXT_BLOCK_SEQ]->(bs)}
    SET bs:First, bs:CurrentFirst
    """,
    """
    MATCH (bs:BlockSeq)
    WHERE NOT EXISTS {(bs)-[:NEXT_BLOCK_SEQ]->()}
    SET bs:Last, bs:CurrentLast
    """
    ]

    for q in queries:
        gds.run_cypher(q, {"block_seq_list":block_seq_list})

In [40]:
ingest(filename)

## part 1

In [ ]:
infer_queries =[
"""
MATCH path=(first:CurrentFirst&File)((ni:File&!Deactivated)-[:NEXT_BLOCK_SEQ]->())*(ffree:FreeSpace&!Deactivated),
  (lfile:File&!Deactivated)
  (()-[:NEXT_BLOCK_SEQ]->(:FreeSpace&!Deactivated))*(last:CurrentLast)
WHERE lfile.len < ffree.len
MATCH (bffree)-[:NEXT_BLOCK_SEQ]->(ffree)-[:NEXT_BLOCK_SEQ]->(affree)
REMOVE lfile:File, first:CurrentFirst, last:CurrentLast
SET lfile:FreeSpace, lfile:CurrentLast, ffree:Deactivated
CREATE (a:BlockSeq:File:CurrentFirst {len: lfile.len, id_number: lfile.id_number})-[:NEXT_BLOCK_SEQ]->(b:BlockSeq:FreeSpace {len: ffree.len - lfile.len})
MERGE (bffree)-[:NEXT_BLOCK_SEQ]->(a)
MERGE (b)-[:NEXT_BLOCK_SEQ]->(affree);
""","""
MATCH path=(first:CurrentFirst&File)((ni:File&!Deactivated)-[:NEXT_BLOCK_SEQ]->())*(ffree:FreeSpace&!Deactivated),
  (lfile:File&!Deactivated)
  (()-[:NEXT_BLOCK_SEQ]->(:FreeSpace&!Deactivated))*(last:CurrentLast)
WHERE lfile.len > ffree.len
REMOVE ffree:FreeSpace, first:CurrentFirst
SET lfile.len = lfile.len - ffree.len, ffree.id_number = lfile.id_number, ffree:File, ffree:CurrentFirst
""","""
MATCH path=(first:CurrentFirst&File)((ni:File&!Deactivated)-[:NEXT_BLOCK_SEQ]->())*(ffree:FreeSpace&!Deactivated),
  (lfile:File&!Deactivated)
  (()-[:NEXT_BLOCK_SEQ]->(:FreeSpace&!Deactivated))*(last:CurrentLast)
WHERE lfile.len = ffree.len
REMOVE ffree:FreeSpace, lfile:File, first:CurrentFirst, last:CurrentLast
SET lfile:FreeSpace, lfile:CurrentLast, ffree:File, ffree:CurrentFirst, ffree.id_number = lfile.id_number
"""
]
infer(infer_queries)


In [34]:
gds.run_cypher("""MATCH path=(first:First&File)((ni:File&!Deactivated)-[:NEXT_BLOCK_SEQ]->())*(ffree:FreeSpace&!Deactivated)
WITH apoc.coll.flatten([x IN ni | [_ IN range(1, x.len) | x.id_number]]) AS fileCompacting
WITH reduce(acc={checksum:0, step:0}, x in fileCompacting| {checksum: acc.checksum + acc.step * x, step: acc.step + 1}) AS checksum_step
RETURN checksum_step.checksum AS part1""")

,part1
0,6332189866718


### Part 2

In [78]:
ingest(filename)

In [79]:
infer_queries =[
"""
MATCH (last:File&!Processed)
WITH last ORDER BY last.ix DESC LIMIT 1
SET last:Processed
WITH last
CALL (last) {
  MATCH (free:FreeSpace&!Deactivated WHERE free.len >= last.len AND free.ix < last.ix)
  WITH free, last ORDER BY free.ix ASC LIMIT 1
  MATCH (bfree)-[:NEXT_BLOCK_SEQ]->(free)-[:NEXT_BLOCK_SEQ]->(afree)
  CREATE (a:BlockSeq:File {len: last.len, id_number: last.id_number, ix:free.ix})-[:NEXT_BLOCK_SEQ]->(b:BlockSeq:FreeSpace {len: free.len - last.len, ix:free.ix})
  MERGE (bfree)-[:NEXT_BLOCK_SEQ]->(a)
  MERGE (b)-[:NEXT_BLOCK_SEQ]->(afree)
  SET free:Deactivated, last:FreeSpace
  REMOVE last:File
}
"""
]
infer(infer_queries)

In [82]:
gds.run_cypher("""MATCH path=SHORTEST 1 (first:First)((ni:!Deactivated)-[:NEXT_BLOCK_SEQ]->())*(last:Last)
WITH apoc.coll.flatten([x IN ni| [_ IN range(1, x.len) | CASE WHEN x:File&Processed THEN x.id_number ELSE 0 END]]) AS fileCompacting
WITH reduce(acc={checksum:0, step:0}, x in fileCompacting| {checksum: acc.checksum + acc.step * x, step: acc.step + 1}) AS checksum_step
RETURN checksum_step.checksum AS part2""")

,part2
0,6353648390778
